In [1]:
import psycopg2
import pandas as pd
import requests
from io import BytesIO, TextIOWrapper

In [2]:
DBNAME = "opportunity_youth"
conn = psycopg2.connect(dbname=DBNAME)

In [38]:
def query(string):
    return pd.read_sql(string, conn)

In [178]:
def data_dictionary():

    response = requests.get('https://www2.census.gov/programs-surveys/acs/tech_docs/pums/data_dict/PUMS_Data_Dictionary_2013-2017.csv')
    content_as_file = BytesIO(response.content)
    csv_file_text = TextIOWrapper(content_as_file, encoding="ISO-8859-1")
    return pd.read_csv(csv_file_text).reset_index()

In [179]:
d_dict = data_dictionary()

def search(string):
    return d_dict[d_dict.C.str.contains(string,  case=False)]

In [27]:
pd.read_sql("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""", conn)

,table_name
0,pums_2017
1,puma_names_2010
2,il_jobs_2017
3,il_geo_xwalk
4,ct_puma_xwalk


In [58]:
puma = query("""SELECT geoid FROM puma_names_2010 WHERE puma_name LIKE '%Rock Island%'""").geoid[0][2:]

rock_island = query(f"""SELECT * from pums_2017 WHERE puma = '{puma}'""")

geo_xwalk = query(f"""SELECT * 
                FROM il_geo_xwalk 
                WHERE ctyname 
                LIKE '%Rock Island%'
                AND tabblk2010
                LIKE '%{puma}%'""")

In [150]:
variables = search('education')

variables = variables.append(search('disability'))

variables = variables.append(search('work'))

variables = variables.append(search('difficulty'))

variables = variables\
.append(search('race')[search('race')\
                       .level_1.isin(['RACAIAN','RACASN',
                                      'RACBLK','RACNH',
                                      'RACPI','RACSOR',
                                      'RACWHT'])])

variables